In [33]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [34]:
import sys
from importlib import reload
reload(sys)
sys.setdefaultencoding('utf-8')

AttributeError: ignored

In [21]:
!pip install opencv-python matplotlib pandas torch torchvision func_to_script accelerate tqdm albumentations pycocotools tensorboard

In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
!unzip gdrive/My\ Drive/UFPR-ALPR.zip

Streaming output truncated to the last 5000 lines.
  inflating: UFPR-ALPR dataset/training/track0050/track0050[21].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[19].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[30].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[17].png  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[15].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[02].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[17].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[12].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[16].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[23].txt  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[02].png  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[04].png  
  inflating: UFPR-ALPR dataset/training/track0050/track0050[10].png  
  inflating: UFPR-ALPR dataset/training

In [ ]:
#!git clone https://github.com/WongKinYiu/yolov7.git

fatal: destination path 'yolov7' already exists and is not an empty directory.


# New section

In [26]:
!unzip gdrive/My\ Drive/yolov7.zip

Archive:  gdrive/My Drive/yolov7.zip
  inflating: yolov7/__init__.py      
   creating: yolov7/__pycache__/
  inflating: yolov7/__pycache__/__init__.cpython-39.pyc  
  inflating: yolov7/__pycache__/dataset.cpython-39.pyc  
  inflating: yolov7/__pycache__/logging.cpython-39.pyc  
  inflating: yolov7/__pycache__/loss.cpython-39.pyc  
  inflating: yolov7/__pycache__/loss_factory.cpython-39.pyc  
  inflating: yolov7/__pycache__/mosaic.cpython-39.pyc  
  inflating: yolov7/__pycache__/plotting.cpython-39.pyc  
  inflating: yolov7/__pycache__/trainer.cpython-39.pyc  
  inflating: yolov7/__pycache__/utils.cpython-39.pyc  
  inflating: yolov7/anchors.py       
  inflating: yolov7/dataset.py       
   creating: yolov7/evaluation/
  inflating: yolov7/evaluation/__init__.py  
   creating: yolov7/evaluation/__pycache__/
  inflating: yolov7/evaluation/__pycache__/__init__.cpython-310.pyc  
  inflating: yolov7/evaluation/__pycache__/__init__.cpython-39.pyc  
  inflating: yolov7/evaluation/__pycache__

In [28]:
!unzip gdrive/My\ Drive/pytorch_accelerated.zip

Archive:  gdrive/My Drive/pytorch_accelerated.zip
  inflating: pytorch_accelerated/__init__.py  
   creating: pytorch_accelerated/__pycache__/
  inflating: pytorch_accelerated/__pycache__/__init__.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/_version.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/callbacks.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/run_config.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/tracking.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/trainer.cpython-39.pyc  
  inflating: pytorch_accelerated/__pycache__/utils.cpython-39.pyc  
  inflating: pytorch_accelerated/_version.py  
  inflating: pytorch_accelerated/callbacks.py  
  inflating: pytorch_accelerated/finetuning.py  
  inflating: pytorch_accelerated/run_config.py  
   creating: pytorch_accelerated/schedulers/
  inflating: pytorch_accelerated/schedulers/__init__.py  
   creating: pytorch_accelerated/schedulers/__pycache__/
  inflat

In [29]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from pathlib import Path
from functools import partial
import os
import cv2
import torch
import random
import shutil
from collections import Counter
import matplotlib.pyplot as plt
import copy
import datetime

from func_to_script import script
from pytorch_accelerated.callbacks import (
    EarlyStoppingCallback,
    SaveBestModelCallback,
    get_default_callbacks,
)

from torch.utils.tensorboard import SummaryWriter
from pytorch_accelerated.schedulers import CosineLrScheduler
from torch.utils.data import Dataset
from yolov7 import create_yolov7_model
from yolov7.dataset import Yolov7Dataset, create_yolov7_transforms, yolov7_collate_fn
from yolov7.evaluation import CalculateMeanAveragePrecisionCallback
from yolov7.loss_factory import create_yolov7_loss
from yolov7.trainer import Yolov7Trainer, filter_eval_predictions
from yolov7.logging import TensorboardCallback
from yolov7.mosaic import MosaicMixupDataset, create_post_mosaic_transform
from yolov7.dataset import create_yolov7_customtransforms

In [30]:
class DataGenerator:
    """ Used to generate license plate cropped images and annotation csv files for character detection
    """

    def __init__(self):
        self.lp_chars_labels = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H",
                                "I","J", "K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]

        self.yolo_original_imgdir = ""
        self.yolo_generated_imgdir = ""
        self.mode = ""
        self.dataset=""
        self.dataset_object_count = {}
        for key in self.lp_chars_labels:
            self.dataset_object_count[key] = 0


    def get_lp_annotations(self, imgs_dir):

        annos = []
        self.mode = imgs_dir.split("/")[-1]
        self.dataset =imgs_dir.split("/")[-2]
        print("mode:",self.mode)
        print("dataset",self.dataset)
        self.yolo_original_imgdir = f"{self.mode}/{self.dataset}"
        if not os.path.isdir(self.yolo_original_imgdir ):
            os.makedirs(self.yolo_original_imgdir)

        with open(f"{self.mode}.csv", 'w+') as file:
            line = f"image_name,class_id,xmin,ymin,xmax,ymax,class_name"
            file.write(line + '\n')

        dir_names = os.listdir(imgs_dir)

        for dir_name in dir_names:
            filenames = os.listdir(f"{imgs_dir}/{dir_name}")
            filenames = [x for x in filenames if x.endswith('png' or 'jpg'or '.jpeg')]
            for filename in filenames:
                img = cv2.imread(f"{imgs_dir}/{dir_name}/{filename}")
                img_h, img_w, _ = img.shape

                img_name = filename.split(".")[0]
                anno_lines = []
                print(f"{imgs_dir}/{dir_name}/{img_name}.txt")
                with open(f"{imgs_dir}/{dir_name}/{img_name}.txt") as file:
                    anno_lines = file.readlines()

                char_data = list(anno_lines[6].split(": ")[1].replace("-",""))[:-1]

                bbox_lp = [int(x) for x in anno_lines[7].split(": ")[1].split(" ") if x!="\n"]

                aspect_ratio = bbox_lp[2]/bbox_lp[3]
                if aspect_ratio > 2.4:
                    # print('height changed')
                    desired_height = bbox_lp[2] / 2.4
                    height_offset = desired_height - bbox_lp[3]
                    if int(bbox_lp[1] - (height_offset/2)) <0:
                        bbox_lp[1] = 0
                    else:
                        bbox_lp[1] = int(bbox_lp[1] - (height_offset/2))
                    if bbox_lp[1] + int(desired_height) > img_h:
                        bbox_lp[1] = img_h - int(desired_height)
                    bbox_lp[3] = int(desired_height)
                    # train_df.iloc[ind, 5] = train_df.iloc[ind, 5] + (height_offset/2)
                elif aspect_ratio < 2.4:
                    # print('weight changed')
                    desired_width = bbox_lp[3] * 2.4
                    width_offset = desired_width - bbox_lp[2]
                    if int(bbox_lp[0] - (width_offset/2)) <0:
                        bbox_lp[0] = 0
                    else:
                        bbox_lp[0] = int(bbox_lp[0] - (width_offset/2))
                    if bbox_lp[0] + int(desired_width) > img_w:
                        bbox_lp[0] = img_w - int(desired_width)
                    bbox_lp[2] = int(desired_width)

                crop_img = img[bbox_lp[1]:bbox_lp[1]+bbox_lp[3],
                                bbox_lp[0]:bbox_lp[0]+bbox_lp[2]]

                cv2.imwrite(f"{self.yolo_original_imgdir}/{img_name}.png", crop_img)

                bbox_char = []
                for i in range(len(char_data)):
                    bbox  = [int(x) for x in anno_lines[8+i].split(": ")[1].split(" ")]
                    self.dataset_object_count[char_data[i]] += 1

                    coco_bbox_format = [self.lp_chars_labels.index(char_data[i]),
                                              bbox[0] - bbox_lp[0],
                                              bbox[1] - bbox_lp[1],
                                              bbox[0] - bbox_lp[0]+ bbox[2],
                                              bbox[1] - bbox_lp[1]+ bbox[3],
                                              char_data[i]]

                    #custom format yolov7 training
                    with open(f"{self.mode}.csv", 'a+') as file:
                        line = f"{self.yolo_original_imgdir}/{img_name}.png," + ','.join(map(str, coco_bbox_format))
                        file.write(line + '\n')

In [31]:
data_gen = DataGenerator()
data_gen.get_lp_annotations("UFPR-ALPR dataset/training")
data_gen.get_lp_annotations("UFPR-ALPR dataset/validation")
data_gen.get_lp_annotations("UFPR-ALPR dataset/testing")


mode: training
dataset UFPR-ALPR dataset
UFPR-ALPR dataset/training/track0039/track0039[21].txt
UFPR-ALPR dataset/training/track0039/track0039[12].txt
UFPR-ALPR dataset/training/track0039/track0039[18].txt
UFPR-ALPR dataset/training/track0039/track0039[29].txt
UFPR-ALPR dataset/training/track0039/track0039[22].txt
UFPR-ALPR dataset/training/track0039/track0039[25].txt
UFPR-ALPR dataset/training/track0039/track0039[14].txt
UFPR-ALPR dataset/training/track0039/track0039[09].txt
UFPR-ALPR dataset/training/track0039/track0039[13].txt
UFPR-ALPR dataset/training/track0039/track0039[17].txt
UFPR-ALPR dataset/training/track0039/track0039[07].txt
UFPR-ALPR dataset/training/track0039/track0039[26].txt
UFPR-ALPR dataset/training/track0039/track0039[08].txt
UFPR-ALPR dataset/training/track0039/track0039[05].txt
UFPR-ALPR dataset/training/track0039/track0039[16].txt
UFPR-ALPR dataset/training/track0039/track0039[04].txt
UFPR-ALPR dataset/training/track0039/track0039[06].txt
UFPR-ALPR dataset/traini

UnicodeDecodeError: ignored

In [ ]:
class_id_to_label = dict(
        enumerate(['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'])
    )
class_label_to_id = {v: k for k, v in class_id_to_label.items()}

def load_UFPR_df(train_annotation_filename,valid_annotation_filename):
    train_df = pd.read_csv(train_annotation_filename)
    valid_df = pd.read_csv(valid_annotation_filename)

    df = pd.concat((train_df, valid_df))
    all_images = sorted(set(df['image_name']))
    image_id_to_image = {i: im for i, im in enumerate(all_images)}
    image_to_image_id = {v: k for k, v, in image_id_to_image.items()}
    # class_id_to_label = dict(
    #     enumerate(df.class_name.unique())
    # )
    # class_label_to_id = {v: k for k, v in class_id_to_label.items()}

    train_df["image_id"] = train_df.image_name.map(image_to_image_id)
    train_df["class_id"] = train_df.class_name.map(class_label_to_id)

    valid_df["image_id"] = valid_df.image_name.map(image_to_image_id)
    valid_df["class_id"] = valid_df.class_name.map(class_label_to_id)

    lookups = {
        "image_id_to_image": image_id_to_image,
        "image_to_image_id": image_to_image_id,
        "class_id_to_label": class_id_to_label,
        "class_label_to_id": class_label_to_id,
    }
    return train_df, valid_df, lookups

class UFPRDatasetAdaptor(Dataset):
    def __init__(
        self,
        images_dir_path,
        annotations_dataframe,
        transforms=None,
    ):
        self.images_dir_path = Path(images_dir_path)
        self.annotations_df = annotations_dataframe
        self.transforms = transforms

        self.image_idx_to_image_id = {
            idx: image_id
            for idx, image_id in enumerate(self.annotations_df.image_id.unique())
        }
        self.image_id_to_image_idx = {
            v: k for k, v, in self.image_idx_to_image_id.items()
        }

    def __len__(self) -> int:
        return len(self.image_idx_to_image_id)

    def __getitem__(self, index):
        image_id = self.image_idx_to_image_id[index]
        image_info = self.annotations_df[self.annotations_df.image_id == image_id]
        file_name = image_info.image_name.values[0]
        assert image_id == image_info.image_id.values[0]

        image = Image.open(self.images_dir_path / file_name).convert("RGB")
        image = np.array(image)

        image_hw = image.shape[:2]

        # if image_info.has_annotation.any():
        xyxy_bboxes = image_info[["xmin", "ymin", "xmax", "ymax"]].values
        class_ids = image_info["class_id"].values

        # else:
        #     xyxy_bboxes = np.array([])
        #     class_ids = np.array([])

        if self.transforms is not None:
            transformed = self.transforms(
                image=image, bboxes=xyxy_bboxes, labels=class_ids
            )
            image = transformed["image"]
            xyxy_bboxes = np.array(transformed["bboxes"])
            class_ids = np.array(transformed["labels"])

        return image, xyxy_bboxes, class_ids, image_id, image_hw

In [ ]:
target_image_size = 640
pretrained = True
num_epochs = 2
batch_size = 4

train_annotation_filename = 'training.csv'
valid_annotation_filename = 'validation.csv'
train_df, valid_df, lookups = load_UFPR_df(train_annotation_filename, valid_annotation_filename)
num_classes = 36
model_name = "model"
logdir = "logs/scalars/" + model_name + '_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = SummaryWriter(logdir)
images_path = Path('./')
train_ds = UFPRDatasetAdaptor(
    images_path, train_df
)
eval_ds = UFPRDatasetAdaptor(
    images_path, valid_df
)

train_yds = Yolov7Dataset(
    train_ds,
    create_yolov7_transforms(training=False,image_size=(target_image_size, target_image_size)),
)
eval_yds = Yolov7Dataset(
    eval_ds,
    create_yolov7_transforms(training=False, image_size=(target_image_size, target_image_size)),
)

# Create model, loss function and optimizer
model = create_yolov7_model(
    architecture="yolov7", num_classes=num_classes, pretrained=pretrained
)
# PATH = "best_model.pt"
# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])

loss_func = create_yolov7_loss(model, image_size=target_image_size)

optimizer = torch.optim.SGD(
    model.parameters(), lr=0.01, momentum=0.9, nesterov=True
)
# Create trainer and train
trainer = Yolov7Trainer(
    model=model,
    optimizer=optimizer,
    loss_func=loss_func,
    filter_eval_predictions_fn=partial(
        filter_eval_predictions, confidence_threshold=0.01, nms_threshold=0.3
    ),
    callbacks=[
        CalculateMeanAveragePrecisionCallback.create_from_targets_df(
            targets_df=valid_df[
                ["image_id", "xmin", "ymin", "xmax", "ymax", "class_id"]
            ],
            image_ids=set(valid_df.image_id.unique()),
            iou_threshold=0.2,
        ),
        SaveBestModelCallback(save_path='UFPR_ch_tuned_model2.pt',watch_metric="map", greater_is_better=True),
        EarlyStoppingCallback(
            early_stopping_patience=3,
            watch_metric="map",
            greater_is_better=True,
            early_stopping_threshold=0.001,
        ),
        TensorboardCallback(tb),
        *get_default_callbacks(progress_bar=True),
    ],
)
trainer.train(
    num_epochs=num_epochs,
    train_dataset=train_yds,
    eval_dataset=eval_yds,
    per_device_batch_size=batch_size,
    create_scheduler_fn=CosineLrScheduler.create_scheduler_fn(
        num_warmup_epochs=5,
        num_cooldown_epochs=5,
        k_decay=2,
    ),
    collate_fn=yolov7_collate_fn,
)

Output hidden; open in https://colab.research.google.com to view.